In [6]:
import numpy as np
import itertools
import pandas as pd  # kept for compatibility with your environment
from numba import njit

# -----------------------------
# Utilities: stable numerics
# -----------------------------

@njit(cache=True)
def _stable_sigmoid_scalar(x):
    # Numerically stable sigmoid for scalar x (Numba-friendly)
    if x >= 0.0:
        return 1.0 / (1.0 + np.exp(-x))
    else:
        ex = np.exp(x)
        return ex / (1.0 + ex)

@njit(cache=True)
def _softplus_scalar(x):
    # Numerically stable softplus: log(1 + exp(x))
    if x > 0.0:
        return x + np.log1p(np.exp(-x))
    else:
        return np.log1p(np.exp(x))

def stable_sigmoid(x):
    # Vectorized stable sigmoid for potential external use (unchanged from your code’s spirit)
    x = np.asarray(x)
    out = np.empty_like(x, dtype=np.float64)
    pos = x >= 0
    neg = ~pos
    out[pos] = 1.0 / (1.0 + np.exp(-x[pos]))
    exp_x = np.exp(x[neg])
    out[neg] = exp_x / (1.0 + exp_x)
    return out

# -----------------------------
# Core heavy kernels (Numba)
# -----------------------------

@njit(cache=True)
def _iterate_combinations_grad_nll(beta, degrees_float, r):
    """
    Stream all r-combinations of n indices without materializing the entire list.
    Compute:
      - grad: sum over combinations containing j of sigmoid(sum(beta_e)) for each j
      - nll: sum softplus(sum(beta_e)) - beta·degrees
    degrees_float must be float64.
    """
    n = beta.shape[0]
    grad = np.zeros(n, dtype=np.float64)
    term1 = 0.0

    # initial combination [0,1,2,...,r-1]
    idx = np.arange(r, dtype=np.int64)

    while True:
        # sum beta over current combination
        s = 0.0
        for j in range(r):
            s += beta[idx[j]]

        sig = _stable_sigmoid_scalar(s)
        sp = _softplus_scalar(s)

        # accumulate gradient contributions
        for j in range(r):
            grad[idx[j]] += sig

        # accumulate NLL term1
        term1 += sp

        # generate next combination
        i = r - 1
        # find rightmost index that can be incremented
        while i >= 0 and idx[i] == i + n - r:
            i -= 1
        if i < 0:
            break
        idx[i] += 1
        for j in range(i + 1, r):
            idx[j] = idx[j - 1] + 1

    # subtract degrees from gradient and compute term2 = beta·degrees
    term2 = 0.0
    for i in range(n):
        grad[i] -= degrees_float[i]
        term2 += beta[i] * degrees_float[i]

    nll = term1 - term2
    return grad, nll


@njit(cache=True)
def _generate_degrees_from_beta(n, r, beta):
    """
    Generate degrees for an r-uniform hypergraph by iterating all r-combinations.
    For each combination e, include it with probability sigmoid(sum(beta_e)).
    Returns degrees (int64). We do not store edges to save memory.
    """
    degrees = np.zeros(n, dtype=np.int64)

    idx = np.arange(r, dtype=np.int64)
    while True:
        s = 0.0
        for j in range(r):
            s += beta[idx[j]]
        p = _stable_sigmoid_scalar(s)
        if np.random.random() < p:
            for j in range(r):
                degrees[idx[j]] += 1

        # next combination
        i = r - 1
        while i >= 0 and idx[i] == i + n - r:
            i -= 1
        if i < 0:
            break
        idx[i] += 1
        for j in range(i + 1, r):
            idx[j] = idx[j - 1] + 1

    return degrees

# -----------------------------
# Public API (compat signatures)
# -----------------------------

def generate_beta(n, rho):
    rng = np.random.default_rng(seed=None)
    # mixture: with prob rho -> 0, else N(0,1)
    mask = rng.random(n) < rho
    betas = np.empty(n, dtype=np.float64)
    betas[mask] = 0.0
    betas[~mask] = rng.normal(size=(~mask).sum())
    return betas

def generate_r_uniform_hypergraph(n, r, beta):
    """
    Kept for compatibility with your call-site.
    We no longer store edges (to avoid huge memory). Return an empty list for edges.
    """
    degrees = _generate_degrees_from_beta(n, r, beta)
    edges = []  # unused in downstream; preserved to keep tuple unpacking
    return edges, degrees

def add_gaussian_noise(degrees, epsilon, delta, r):
    c_squared = 2.0 * np.log(1.25 / delta)
    sigma = np.sqrt(c_squared) * np.sqrt(r) / epsilon
    noise = np.random.normal(0.0, sigma, size=degrees.shape)
    return degrees + noise

def neg_log_likelihood(beta, degrees, r):
    """
    Provided for completeness; uses the same streamed exact computation.
    """
    deg_float = degrees.astype(np.float64, copy=False)
    _, nll = _iterate_combinations_grad_nll(beta, deg_float, r)
    return nll

In [7]:
def gradient_descent(n, r, noisy_degrees, lr=0.01, max_iter=1000,
                     tol=1e-4, verbose=True):
    
    beta = np.zeros(n, dtype=np.float64)
    degrees_f = noisy_degrees.astype(np.float64, copy=False)
    success = 0

    for t in range(max_iter):
        grad, nll = _iterate_combinations_grad_nll(beta, degrees_f, r)
        grad_norm = np.linalg.norm(grad)

        if grad_norm < tol:
            print(f"Iter {t}: grad norm = {grad_norm:.4f}, NLL = {nll:.4f}")
            success = 1
            break

        beta -= lr * grad

        if verbose and (t % 10 == 0):
            print(f"Iter {t}: grad norm = {grad_norm:.4f}, NLL = {nll:.4f}")

    return beta, success

def run_simulation(n, r, rho, delta, epsilon):
    beta = generate_beta(n, rho)
    _, degrees = generate_r_uniform_hypergraph(n, r, beta)

    lr = 0.000005
    max_iter = 200
    l2_error = []
    linf_error = []

    if epsilon is None:
        beta_hat_0, success = gradient_descent(n, r, degrees, lr, max_iter)
        l2 = np.linalg.norm(beta - beta_hat_0)
        l2_error.append(l2)
        linf = np.abs(beta - beta_hat_0).max()
        linf_error.append(linf)
        print(l2, linf)
        return l2_error, linf_error, success

    noisy_degrees = add_gaussian_noise(degrees, epsilon, delta, r)
    beta_hat, success = gradient_descent(n, r, noisy_degrees, lr, max_iter)
    l2 = np.linalg.norm(beta - beta_hat)
    l2_error.append(l2)
    linf = np.abs(beta - beta_hat).max()
    linf_error.append(linf)
    print(l2, linf)
    return l2_error, linf_error, success

delta = 1e-5
rho = 0.6
r = 3
e_list = [None, 2, 1.5, 1, 0.5, 0.1]
result = []
# e_list = [None]

#100 done
for n in [100, 200, 300]:
    for e in e_list:
        print(f"\n Running for r = {r}, n = {n}, epsilon = {e} -> \n")
        with open(f"l_2 error, n - {n}, r - {r}, e - {e}.csv", 'a') as f1, \
             open(f"l_inf error, n - {n}, r - {r}, e - {e}.csv", 'a') as f2:
            c = 0
            t = 10
            while (c < 5 and t > 0):
                l2, linf, success = run_simulation(n, r, rho, delta, e)
                t -= 1
                if success == 1:
                    c += 1
                    # Write current row to both files (same filenames/format)
                    f1.write(','.join(map(str, l2)) + '\n')
                    f2.write(','.join(map(str, linf)) + '\n')
                # keep your tracking print
                print("t = ", 10 - t, "vs c = ", c) #
        g = [r, n, e, 10 - t, c]
        print(g)
        result.append(g)


 Running for r = 3, n = 1000, epsilon = 2 -> 

Iter 0: grad norm = 1933517.7965, NLL = 115178187.7333
Iter 10: grad norm = 30963.8481, NLL = 98237812.8229
Iter 20: grad norm = 4545.5128, NLL = 98225803.0812
Iter 30: grad norm = 1128.5497, NLL = 98225445.2541
Iter 40: grad norm = 336.8450, NLL = 98225420.1858
Iter 50: grad norm = 107.6670, NLL = 98225417.8367
Iter 60: grad norm = 35.2511, NLL = 98225417.5923
Iter 70: grad norm = 11.6350, NLL = 98225417.5659
Iter 80: grad norm = 3.8505, NLL = 98225417.5631
Iter 90: grad norm = 1.2754, NLL = 98225417.5628
Iter 100: grad norm = 0.4226, NLL = 98225417.5627
Iter 110: grad norm = 0.1400, NLL = 98225417.5627
Iter 120: grad norm = 0.0464, NLL = 98225417.5628
Iter 130: grad norm = 0.0154, NLL = 98225417.5628
Iter 140: grad norm = 0.0051, NLL = 98225417.5628
Iter 150: grad norm = 0.0017, NLL = 98225417.5628
Iter 160: grad norm = 0.0006, NLL = 98225417.5628
Iter 170: grad norm = 0.0002, NLL = 98225417.5628
Iter 176: grad norm = 0.0001, NLL = 9822